In [183]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget.neurons import adex
from voltagebudget.util import read_args
from voltagebudget.util import read_stim
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import mad
from voltagebudget.util import write_spikes
from voltagebudget.util import filter_voltages

Loading BokehJS ...

In [126]:
# ---------------------------------------------------------------
stim = "../data/stim1.csv"
verbose = True

# ---------------------------------------------------------------
if verbose:
    print(">>> Importing stimulus from {}.".format(stim))

stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

# ---------------------------------------------------------------
f = 20
N = 1
t = 0.4
time_step = 1e-5
seed_value = 42

mode = 'regular'
params, w_in, bias_in, sigma = util.read_modes(mode)

>>> Importing stimulus from ../data/stim1.csv.


In [248]:
A_ref = 0.0

ns_ref, ts_ref, voltages_ref = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    E=0.001,
    n_cycles=0,
    f=f,
    A=A_ref,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [260]:
A_shift = 0.01e-9

ns_shift, ts_shift, voltages_shift = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    E=0.001,
    n_cycles=10,
    f=f,
    A=A_shift,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [253]:
A_add = 0.025e-9

ns_add, ts_add, voltages_add = adex(
    N,
    t,
    ns,
    ts,
    w_in=w_in,
    bias_in=bias_in,
    E=0.001,
    n_cycles=10,
    f=20,
    A=A_add,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    seed_value=seed_value,
    **params)

In [261]:
p = figure(plot_width=400, plot_height=200, toolbar_location=None)

p.vbar(x=ts_ref, width=0.00005, 
       bottom=np.zeros_like(ts_ref), top=np.ones_like(ts_ref), 
       color="black", alpha=0.5)
p.vbar(x=ts_shift, width=0.00005, 
       bottom=np.zeros_like(ts_shift), top=np.ones_like(ts_shift), 
       color="red", alpha=0.7)
p.vbar(x=ts_add, width=0.00005, 
       bottom=np.zeros_like(ts_add), top=np.ones_like(ts_add), 
       color="red", alpha=0.3)

p.xaxis.axis_label = 'Times (s)'
p.yaxis.axis_label = 'N'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(mad(ts_ref), mad(ts_shift), mad(ts_add))
print(ts_ref.size, ts_shift.size, ts_add.size)

(0.007923749999999986, 0.009492500000000001, 0.011628799999999995)
(4, 4, 5)


# Save

- Ref, shift, add

In [262]:
times = voltages_ref['times']
v_c = voltages_ref['V_comp'][0, :]
v_o = voltages_ref['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_ref_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")

write_spikes("../analysis/spks_ref", ns_ref, ts_ref)

In [263]:
times = voltages_shift['times']
v_c = voltages_shift['V_comp'][0, :]
v_o = voltages_shift['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_shift_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")

write_spikes("../analysis/spks_shift", ns_shift, ts_shift)

In [264]:
times = voltages_add['times']
v_c = voltages_add['V_comp'][0, :]
v_o = voltages_add['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_add_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")

write_spikes("../analysis/spks_add", ns_add, ts_add)